<a href="https://colab.research.google.com/github/danielelbrecht/CAP5610-HW-1/blob/master/CAP5610_HW1Problem3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Single layer digit classifier using softmax activation and categorical cross entropy loss

In [0]:
#Import data
import numpy as np
import sklearn
from keras.datasets import mnist

#Load data
(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()

In [0]:
#Preprocess data
train_images = train_images_original.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images_original.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255



#Use one-hot encoding
from keras.utils import to_categorical

train_labels = to_categorical(train_labels_original)
test_labels = to_categorical(test_labels_original)

In [0]:


#Returns the ativations for an entire mini batch
def softmax(weights, data):
  
  activations = []
  
  for i in range(len(data)):
    total = 0
    temp = []
    for j in range(10):
      total = total + np.exp(np.dot(weights[j], data[i]))
      temp.append(np.exp(np.dot(weights[j], data[i])))
    temp = temp / total
    activations.append(temp)
    
  return activations
  
  


#gradient function for softmax activation with log loss = y(a-k)*X
def gradient(weights, bias, x, y):
  
  #Get softmax activations for mini batch
  activations = softmax(weights, x)
  
  #Calculate dL/dz
  #Calculate each partial derivative
  for i in range(10):
    
    
    #Subtract Kronecker Delta
    for j in range(len(activations)):
      activations[j][i] = activations[j][i]-1

  z = np.multiply(y, activations)
 
  
  return np.dot(np.transpose(z), x)

def bias_gradient():
  return

#Calculat cce loss
def cce_loss(weights, bias, x, y):
  
  length = len(y)
  loss = 0
  predictions = softmax(weights, data)
  
  for i in range(length):
    temp = 0
    for j in range(10):
      temp = temp + y[i][j] * np.log(predictions[i][j])
      
    loss = loss - temp
      
  return loss / float(length)

In [0]:
import random



#Initialize weight matrix
#w[0][50] represents the 50th weight of the 0th neuron
weights = []
bias = []

for i in range(10):
  weights.append([])
  bias.append(random.random()-0.5)
  for j in range(784):
    weights[i].append(random.random()-0.5)
    

In [115]:
#Testing
activations = softmax(weights, train_images[2:10])
print(activations[0])
print(train_labels[0])

[0.00129354 0.00322293 0.03930194 0.33505381 0.0004036  0.03187763
 0.29542617 0.28246344 0.0020157  0.00894123]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [116]:
#Training

#Define hyperparameters
epochs = 1
batch_size = 1000
learning_rate = 0.01
length = len(train_images)


#Iterate over epochs
for i in range(epochs):
  
  #Iterate over mini batches
  for j in range(int(length/batch_size)):
    
    
    #get mini batch
    data = train_images[batch_size*j:min(length, batch_size*(j+1))]
    labels = train_labels[batch_size*j:min(length, batch_size*(j+1))]
    
    #Update weights
    weights = weights - learning_rate*gradient(weights, bias, data, labels)
    #bias = bias - learning_rate * bias_gradient(weight, bias, data, labels)
    
    print("Mini batch %i complete" % j)
    
    #measure loss after every 10,000 data points
    if batch_size * j % 10000 == 0:
      print("Loss = %i" % cce_loss(weights, bias, data, labels))
      
  print('Epoch %i complete' % i)

Mini batch 0 complete
Loss = 32
Mini batch 1 complete
Mini batch 2 complete
Mini batch 3 complete
Mini batch 4 complete
Mini batch 5 complete
Mini batch 6 complete
Mini batch 7 complete
Mini batch 8 complete
Mini batch 9 complete
Mini batch 10 complete
Loss = 35
Mini batch 11 complete
Mini batch 12 complete
Mini batch 13 complete
Mini batch 14 complete
Mini batch 15 complete
Mini batch 16 complete
Mini batch 17 complete
Mini batch 18 complete
Mini batch 19 complete
Mini batch 20 complete
Loss = 32
Mini batch 21 complete
Mini batch 22 complete
Mini batch 23 complete
Mini batch 24 complete
Mini batch 25 complete
Mini batch 26 complete
Mini batch 27 complete
Mini batch 28 complete
Mini batch 29 complete
Mini batch 30 complete
Loss = 31
Mini batch 31 complete
Mini batch 32 complete
Mini batch 33 complete
Mini batch 34 complete
Mini batch 35 complete
Mini batch 36 complete
Mini batch 37 complete
Mini batch 38 complete
Mini batch 39 complete
Mini batch 40 complete
Loss = 32
Mini batch 41 com

In [117]:
predictions = softmax(weights, train_images[1:20])

for i in range(10):
  print(predictions[i])
  print(np.argmax(train_labels[i]))
  print()

[9.89632552e-26 8.35288350e-18 5.15255647e-18 6.00680312e-15
 4.62162937e-15 1.39648571e-18 1.18043318e-16 1.20266203e-17
 8.09138522e-17 1.00000000e+00]
5

[2.62461705e-13 1.95533536e-09 2.64423352e-09 2.72384413e-07
 3.22824995e-10 2.91784641e-09 2.50713019e-08 4.25338651e-08
 1.80782673e-09 9.99999650e-01]
0

[6.50474387e-13 1.06222818e-10 1.08532904e-09 4.21901324e-10
 1.28829450e-09 3.62622192e-11 5.59691151e-10 8.14472463e-11
 1.79914176e-11 9.99999996e-01]
4

[8.28424889e-23 4.43528683e-20 1.50601232e-20 3.16791319e-18
 5.92332580e-19 1.05611707e-20 6.46392125e-20 1.10063815e-19
 7.61639852e-21 1.00000000e+00]
1

[1.22544878e-24 2.14903478e-18 1.80528444e-20 1.12694069e-18
 5.22586095e-18 5.34707249e-20 1.64641332e-18 9.30623924e-18
 3.35612097e-20 1.00000000e+00]
9

[3.34843111e-15 8.08014921e-13 2.15487789e-11 9.72541860e-13
 1.16202732e-12 5.42639517e-12 2.34003482e-13 2.02897734e-12
 2.96239560e-12 1.00000000e+00]
2

[1.03264501e-29 2.76996300e-21 3.46996253e-22 9.88910481e-